In [2]:
library(tidyverse)
library(skimr)
library(lubridate)
library(lightgbm)
library(xgboost)
library(h2o)
library(catboost)
library(caret)
library(Metrics)

In [3]:
train <- read_csv('../input/mh-dare-in-reality-2021//train.csv')
tr_weather <- read_csv('../input/mh-dare-in-reality-2021//train_weather.csv')
test <- read_csv('../input/mh-dare-in-reality-2021//test.csv')
te_weather <- read_csv('../input/mh-dare-in-reality-2021//test_weather.csv')
sample_sub <- read_csv("../input/mh-dare-in-reality-2021//submission.csv")

In [4]:
TEAM_NAME <- tibble(
  NUMBER = c(4,37,5,17,
             6,7,8,88,
             10,20,11,33,
             13,25,22,23,
             27,28,29,94,
             36,99,48,71),
  TEAM = c("EVR","EVR","MEQ","MEQ",
           "DRA","DRA","NIO","NIO",
           "JR","JR","AUDI","AUDI",
           "DS","DS","NIS","NIS",
           "BMW","BMW","MAH","MAH",
           "TAG","TAG","ROK","ROK"))

In [5]:
train <- train %>% select(-TEAM) %>% left_join(.,TEAM_NAME,by="NUMBER")
test <- test %>% select(-TEAM) %>% left_join(.,TEAM_NAME,by="NUMBER")

In [6]:
te_weather <- te_weather %>% mutate(RAIN = as.character(RAIN)) %>%
  rename(c("EVENT"="EVENTS"))

In [7]:
weather <- bind_rows(tr_weather,te_weather)

In [8]:
event <- weather %>% 
  separate(TIME_UTC_STR,
           c("TIME_UTC_DATE","TIME_UTC_TIME"),
           sep=" ",remove=FALSE) %>%
  mutate(TIME_UTC_DATE = dmy(TIME_UTC_DATE),
         TIME_UTC_TIME = hm(TIME_UTC_TIME),
         TIME_UTC_STR = dmy_hm(TIME_UTC_STR)) %>%
  group_by(EVENT,LOCATION,TIME_UTC_DATE) %>%
  summarise(MIN_TIM = min(TIME_UTC_SECONDS),MAX_TIM = max(TIME_UTC_SECONDS)) %>%
  mutate(SESSION_TIME = (MAX_TIM-MIN_TIM),
         RACES = n()) %>%
  arrange(EVENT,LOCATION,TIME_UTC_DATE) %>%
  mutate(ROW_N=row_number())

In [9]:

# Cleaning Weather data

w_pressure <- weather %>% distinct(PRESSURE) %>%
  mutate(PR =floor(PRESSURE)) %>%
  mutate(PRESSURE_2 = case_when(PR>770000&PR<777000~PR/1000,
                                PR>100000&PR<120000~PR/100,
                                PR>10000&PR<11000~PR/10,
                                PR>77000&PR<777000~PR/100,
                                TRUE~PR))


In [10]:

weather_clean <- weather %>%
  mutate(WIND_SPEED = as.numeric(substr(str_replace(WIND_SPEED,",","."),1,4)),
         RAIN=str_replace(RAIN,",","."),
         AIR_TEMP = as.integer(substr(as.character(round(AIR_TEMP,0)),1,2)),
         TRACK_TEMP = as.numeric(substr(as.character(round(TRACK_TEMP,0)),1,2)),
         HUMIDITY = as.numeric(substr(as.character(round(HUMIDITY,0)),1,2))
  ) %>% 
  left_join(.,select(w_pressure,PRESSURE,PRESSURE_2),by="PRESSURE") %>%
  select(-PRESSURE) %>%
  mutate(TIME_UTC_STR = dmy_hm(TIME_UTC_STR))


In [11]:

w_index2 <- weather %>%
  ungroup() %>%
  separate(TIME_UTC_STR,
           c("TIME_UTC_DATE","TIME_UTC_TIME"),
           sep=" ",remove=FALSE) %>%
  mutate(TIME_UTC_DATE = dmy(TIME_UTC_DATE),
         TIME_UTC_TIME = hm(TIME_UTC_TIME),
         TIME_UTC_STR = dmy_hm(TIME_UTC_STR)) %>%
  select(TIME_UTC_STR,EVENT,LOCATION,TIME_UTC_DATE) %>% 
  group_by(EVENT,LOCATION,TIME_UTC_DATE) %>%
  summarise(MIN_DT = min(TIME_UTC_STR)) %>%
  arrange(EVENT,LOCATION) %>%
  mutate(ROW_N=row_number()) %>%
  nest(MIN_DT) %>%
  mutate(TIME_UTC_STR = map(data,
                            ~seq.POSIXt(from = .x$MIN_DT,
                                        by="min",
                                        length.out = 60))) %>%
  unnest(TIME_UTC_STR) %>%
  mutate(DATE = date(TIME_UTC_STR)) %>%
  group_by(EVENT,LOCATION,DATE) %>%
  mutate(T_IDX = row_number()) 


In [12]:

tr_wd <- weather %>%
  ungroup() %>%
  separate(TIME_UTC_STR,
           c("TIME_UTC_DATE","TIME_UTC_TIME"),
           sep=" ",remove=FALSE) %>%
  mutate(TIME_UTC_DATE = dmy(TIME_UTC_DATE),
         TIME_UTC_TIME = hm(TIME_UTC_TIME),
         TIME_UTC_STR = dmy_hm(TIME_UTC_STR)) %>%
  #filter(TIME_UTC_DATE!="2021-08-13") %>%
  group_by(EVENT,LOCATION,TIME_UTC_DATE) %>%
  summarise(MIN_TIM = min(TIME_UTC_SECONDS)) %>%
  mutate(RACES = n()) %>%
  arrange(EVENT,LOCATION) %>%
  mutate(ROW_N=row_number()) %>%
  select(EVENT,LOCATION,TIME_UTC_DATE,ROW_N)


In [13]:

#now joining all data together

w_final <- w_index2 %>% 
  left_join(.,weather_clean,by=c("EVENT","LOCATION","TIME_UTC_STR")) %>%
  group_by(EVENT,LOCATION) %>%
  mutate_at(vars(AIR_TEMP,TRACK_TEMP,HUMIDITY,PRESSURE_2),
            ~ifelse(is.na(.x), mean(.x, na.rm = TRUE), .x)) %>% #mean imputation
  mutate_at(vars(TIME_UTC_SECONDS),
            ~ifelse(is.na(.x), as.numeric(TIME_UTC_STR), .x)) %>% 
  fill(WIND_SPEED,WIND_DIRECTION,RAIN,PRESSURE_2,T_IDX,.direction="down") %>%
  select(-data) %>% mutate(T_SEC = T_IDX*60) 


In [14]:

#### driver data

train <- train %>% 
  ungroup() %>%
  group_by(EVENT,LOCATION,NUMBER,LAP_NUMBER) %>%
  mutate(ROW_N = row_number())

test <- test %>% 
  ungroup() %>%
  group_by(EVENT,LOCATION,NUMBER,LAP_NUMBER) %>%
  mutate(ROW_N = row_number())



In [15]:

tr_sector <- train %>%
  select(EVENT,LOCATION,NUMBER,LAP_NUMBER,LAP_TIME,S1_LARGE,S2_LARGE,
         S3_LARGE,PIT_TIME,CROSSING_FINISH_LINE_IN_PIT,ROW_N) %>%
  group_by(EVENT,LOCATION,NUMBER) %>%
  mutate(TOTAL_LAPS = n()) %>% ungroup() %>%
  gather(.,"SESSION","TIME",6:9) %>%
  arrange(EVENT,LOCATION,ROW_N,NUMBER,LAP_NUMBER,SESSION) %>%
  # here the pit time come times combines added to S3 of previous lap or S1 lap where the pit time is generated. 
  # majority we see that the pit time is added to S1, very few corner cases we see in S3
  group_by(EVENT,LOCATION,NUMBER) %>%
  separate(TIME,c("MIN","SEC"),sep=":",remove=FALSE,extra="drop") %>%
  mutate(TOTAL=(as.integer(MIN)*60)+as.integer(SEC)) %>%
  mutate(PREV_SESS = lag(SESSION),NXT_SESS = lead(SESSION), 
         PREV_TM = lag(TOTAL), NXT_TM = lead(TOTAL)) %>% 
  mutate(PREV_FL = lag(CROSSING_FINISH_LINE_IN_PIT,4),
         NXT_FL = lead(CROSSING_FINISH_LINE_IN_PIT,4)) %>%
  replace_na(list(MIN=0,SEC=0,TOTAL=0,PREV_TM=0,NXT_TM=0)) %>% 
  arrange(EVENT,LOCATION,ROW_N,NUMBER,LAP_NUMBER,SESSION) %>%
  group_by(EVENT,LOCATION,NUMBER) %>%
  mutate(ACTUAL_TIME = case_when(LAP_NUMBER==1&SESSION=="S1_LARGE"&PREV_SESS=="PIT_TIME"~TOTAL-PREV_TM,
                                 CROSSING_FINISH_LINE_IN_PIT=="B"&SESSION=="S3_LARGE"&(TOTAL-NXT_TM)>0~TOTAL-NXT_TM,
                                 CROSSING_FINISH_LINE_IN_PIT=="B"&SESSION=="S3_LARGE"&(TOTAL-NXT_TM)<0~TOTAL,
                                 LAP_NUMBER!=TOTAL_LAPS&SESSION=="S1_LARGE"&PREV_SESS=="PIT_TIME"&(TOTAL-PREV_TM)<0~TOTAL,
                                 LAP_NUMBER!=TOTAL_LAPS&SESSION=="S1_LARGE"&PREV_SESS=="PIT_TIME"&(TOTAL-PREV_TM)>0~TOTAL-PREV_TM,
                                 LAP_NUMBER==TOTAL_LAPS~TOTAL,
                                 TRUE~TOTAL)) %>%
  mutate(CUML_TM = cumsum(ACTUAL_TIME)) %>%
  mutate(CUML_TM_MIN = ceiling(CUML_TM/60))



In [16]:

te_sector <- test %>%
  select(EVENT,LOCATION,NUMBER,LAP_NUMBER,LAP_TIME,S1_LARGE,S2_LARGE,
         S3_LARGE,PIT_TIME,CROSSING_FINISH_LINE_IN_PIT,ROW_N) %>%
  group_by(EVENT,LOCATION,NUMBER) %>%
  mutate(TOTAL_LAPS = n()) %>% ungroup() %>%
  gather(.,"SESSION","TIME",6:9) %>%
  arrange(EVENT,LOCATION,ROW_N,NUMBER,LAP_NUMBER,SESSION) %>%
  # here the pit time come times combines added to S3 of previous lap or S1 lap where the pit time is generated. 
  # majority we see that the pit time is added to S1, very few corner cases we see in S3
  group_by(EVENT,LOCATION,NUMBER) %>%
  separate(TIME,c("MIN","SEC"),sep=":",remove=FALSE,extra="drop") %>%
  mutate(TOTAL=(as.integer(MIN)*60)+as.integer(SEC)) %>%
  mutate(PREV_SESS = lag(SESSION),NXT_SESS = lead(SESSION), 
         PREV_TM = lag(TOTAL), NXT_TM = lead(TOTAL)) %>% 
  mutate(PREV_FL = lag(CROSSING_FINISH_LINE_IN_PIT,4),
         NXT_FL = lead(CROSSING_FINISH_LINE_IN_PIT,4)) %>%
  replace_na(list(MIN=0,SEC=0,TOTAL=0,PREV_TM=0,NXT_TM=0)) %>% 
  arrange(EVENT,LOCATION,ROW_N,NUMBER,LAP_NUMBER,SESSION) %>%
  group_by(EVENT,LOCATION,NUMBER) %>%
  mutate(ACTUAL_TIME = case_when(LAP_NUMBER==1&SESSION=="S1_LARGE"&PREV_SESS=="PIT_TIME"~TOTAL-PREV_TM,
                                 CROSSING_FINISH_LINE_IN_PIT=="B"&SESSION=="S3_LARGE"&(TOTAL-NXT_TM)>0~TOTAL-NXT_TM,
                                 CROSSING_FINISH_LINE_IN_PIT=="B"&SESSION=="S3_LARGE"&(TOTAL-NXT_TM)<0~TOTAL,
                                 LAP_NUMBER!=TOTAL_LAPS&SESSION=="S1_LARGE"&PREV_SESS=="PIT_TIME"&(TOTAL-PREV_TM)<0~TOTAL,
                                 LAP_NUMBER!=TOTAL_LAPS&SESSION=="S1_LARGE"&PREV_SESS=="PIT_TIME"&(TOTAL-PREV_TM)>0~TOTAL-PREV_TM,
                                 LAP_NUMBER==TOTAL_LAPS~TOTAL,
                                 TRUE~TOTAL)) %>%
  mutate(CUML_TM = cumsum(ACTUAL_TIME)) %>%
  mutate(CUML_TM_MIN = ceiling(CUML_TM/60))


In [17]:

tr_sec_final <- tr_sector %>%
  select(EVENT,LOCATION,ROW_N,NUMBER,LAP_NUMBER,SESSION,CUML_TM_MIN) %>%
  left_join(.,w_final,by=c("EVENT","LOCATION","ROW_N","CUML_TM_MIN"="T_IDX")) %>%
  arrange(EVENT,LOCATION,ROW_N,NUMBER,LAP_NUMBER) %>%
  select(-c(TIME_UTC_SECONDS,T_SEC))

te_sec_final <- te_sector %>%
  select(EVENT,LOCATION,ROW_N,NUMBER,LAP_NUMBER,SESSION,CUML_TM_MIN) %>%
  left_join(.,w_final,by=c("EVENT","LOCATION","ROW_N","CUML_TM_MIN"="T_IDX")) %>%
  arrange(EVENT,LOCATION,ROW_N,NUMBER,LAP_NUMBER) %>%
  select(-c(TIME_UTC_SECONDS,T_SEC))


In [18]:

#retransform the data 
tr_sec_wide <- tr_sec_final %>%
  pivot_wider(.,
             id_cols = c(EVENT,LOCATION,NUMBER,LAP_NUMBER,ROW_N,TIME_UTC_DATE),
             names_from = SESSION,
             values_from = c(AIR_TEMP,TRACK_TEMP,HUMIDITY,PRESSURE_2,
                             WIND_SPEED,WIND_DIRECTION,RAIN))

te_sec_wide <- te_sec_final %>%
  pivot_wider(.,
              id_cols = c(EVENT,LOCATION,NUMBER,LAP_NUMBER,ROW_N,TIME_UTC_DATE),
              names_from = SESSION,
              values_from = c(AIR_TEMP,TRACK_TEMP,HUMIDITY,PRESSURE_2,
                              WIND_SPEED,WIND_DIRECTION,RAIN))


In [19]:

tr_sector_time <- tr_sector %>%
  select(EVENT,LOCATION,NUMBER,LAP_NUMBER,ROW_N,SESSION,ACTUAL_TIME) %>%
  pivot_wider(.,
              id_cols = c(EVENT,LOCATION,NUMBER,LAP_NUMBER,ROW_N),
              names_from = SESSION,
              values_from = ACTUAL_TIME)


te_sector_time <- te_sector %>%
  select(EVENT,LOCATION,NUMBER,LAP_NUMBER,ROW_N,SESSION,ACTUAL_TIME) %>%
  pivot_wider(.,
              id_cols = c(EVENT,LOCATION,NUMBER,LAP_NUMBER,ROW_N),
              names_from = SESSION,
              values_from = ACTUAL_TIME)


In [20]:
summary(tr_sector_time)

In [21]:

tr_clean <- train %>% select(EVENT,LOCATION,NUMBER,LAP_NUMBER,ROW_N,LAP_TIME) %>%
  inner_join(.,tr_sec_wide,by=c("EVENT","LOCATION","NUMBER","LAP_NUMBER","ROW_N")) %>%
  inner_join(.,tr_sector_time,by=c("EVENT","LOCATION","NUMBER","LAP_NUMBER","ROW_N"))
  

te_clean <- test %>% select(EVENT,LOCATION,NUMBER,LAP_NUMBER,ROW_N,LAP_TIME) %>%
  inner_join(.,te_sec_wide,by=c("EVENT","LOCATION","NUMBER","LAP_NUMBER","ROW_N")) %>%
  inner_join(.,te_sector_time,by=c("EVENT","LOCATION","NUMBER","LAP_NUMBER","ROW_N"))


In [22]:
summary(tr_clean)

In [23]:
tr_clean_na <- tr_clean %>% drop_na()
str(tr_clean_na)

In [24]:
#numerical and non numerical data
tr_clean_na <- tr_clean_na %>% ungroup()  %>% drop_na(TIME_UTC_DATE) %>% mutate_at(vars(c(EVENT,LOCATION,NUMBER,LAP_NUMBER)),as.factor) %>%
mutate_at(vars(starts_with("RAIN")),as.numeric) %>% 
mutate_at(vars(contains(c("AIR","WIND","TRACK","PRESSURE","HUMIDIY","RAIN"))),scale) %>% replace(.,is.na(.),-999) 

In [25]:
te_clean <- te_clean %>% ungroup() %>% drop_na(TIME_UTC_DATE) %>% mutate_at(vars(c(EVENT,LOCATION,NUMBER,LAP_NUMBER)),as.factor) %>%
mutate_at(vars(starts_with("RAIN")),as.numeric) %>% 
mutate_at(vars(contains(c("AIR","WIND","TRACK","PRESSURE","HUMIDIY","RAIN"))),scale) %>% replace(.,is.na(.),-999) 

In [27]:
summary(tr_clean_na)

In [28]:
CAT_COLS <- c("EVENT","LOCATION","NUMBER","LAP_NUMBER")
NUM_COLS <- colnames(tr_clean)[-c(1:7)]
Y <- "LAP_TIME"
X <- c(NUM_COLS,CAT_COLS)
X
CAT_COLS

In [29]:
splits <- rsample::initial_split(tr_clean_na,prop = 0.8)
tr_ <- rsample::training(splits)
va_ <- rsample::testing(splits)

In [30]:
#Creating Catboost data
tr_pool <- catboost.load_pool(data =  tr_[,X],label = unlist(tr_[,Y]),cat_features=CAT_COLS)
va_pool <- catboost.load_pool(data =  va_[,X],label = unlist(va_[,Y]),cat_features=CAT_COLS)
te_pool <- catboost.load_pool(data =  te_clean[,X],cat_features=CAT_COLS)

In [29]:
params <- list(iterations=5000,
               learning_rate=0.01,
               depth=8,
               loss_function="RMSE",
               random_seed = 55,
               l2_leaf_reg=10,
               use_best_model=TRUE,
              verbose=FALSE)

In [33]:
model_train <- catboost.train(tr_pool,va_pool,params=params)

In [41]:
feat_imp <- as_tibble(catboost.get_feature_importance(model_train))%>% 
    rownames_to_column() %>% cbind(.,X) %>% arrange(desc(V1))
feat_imp

In [35]:
#Validation set rmlse
va_pred <- catboost.predict(model_train, 
                               va_pool)
rmsle(va_$LAP_TIME,va_pred)

In [36]:
saveRDS(model_train,"catboost_tr_va_20211115_0223AM.rds")

In [37]:
train_pool <- catboost.load_pool(data =  tr_clean[,X],label = unlist(tr_clean[,Y]),cat_features=CAT_COLS)

In [42]:
#the model shrinks after 320
final_params <- list(iterations=320,
               learning_rate=0.01,
               depth=8,
               loss_function="RMSE",
               random_seed = 55,
               l2_leaf_reg=10,
               use_best_model=TRUE)

In [43]:
model_final_train <- catboost.train(train_pool,params=final_params)

In [ ]:
saveRDS(model_final_train,"catboost_final_v1_20211115_0233AM.rds")

In [51]:
te_pred <- catboost.predict(model_final_train,te_pool)
te_pred

In [52]:
te_pred2 <- catboost.predict(model_train,te_pool)
te_pred2

In [53]:
submission <- data.frame(LAP_TIME=te_pred)
filename <- paste('catboost_final_baseline-320',format(Sys.time(),"%Y%m%d%H%M%s"),sep = '_')
write.csv(submission,paste0(filename,'.csv',collapse = ''),row.names = FALSE)

In [ ]:
submission <- data.frame(LAP_TIME=te_pred2)
filename <- paste('catboost_final_baseline-320',format(Sys.time(),"%Y%m%d%H%M%s"),sep = '_')
write.csv(submission,paste0(filename,'.csv',collapse = ''),row.names = FALSE)